In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [2]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [3]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt

In [23]:
# convert_date 실행 예시
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [4]:
def get_data(driver):
    iteration = True
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    try:
        time.sleep(20)
        #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
        # 더보기 클릭
        driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()

        time.sleep(5)
        
        # 숙박 날짜 가져오기
        dates = driver.find_elements_by_class_name("_34Xs-BQm")
        stay_dates = []
        for d in dates:
            stay_date = convert_date(d.text)
            stay_dates.append(stay_date)
            # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
            if stay_date < datetime.date(2018,6,1): 
                iteration = False

        # 닉네임 가져오기
        nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
        # 별점 가져오기
        rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
        scores = []
        for rc in rating_codes:
            cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
            score = int(cls_attr[1])
            scores.append(score)

        # 리뷰 가져오기
        reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")
        
        for i in range(len(nicknames)):
            df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
            #print(df.loc[i])

    except ElementClickInterceptedException as e:
        print(e)
    except StaleElementReferenceException as e:
        print(e)

    finally:
        return df, iteration

In [5]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test_sillastay_mapo.csv', encoding="utf-8-sig")
    
    return data

In [6]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    # 날짜 수정 - 코로나 전 1년 반 - 코로나 후 1년 반
    data = data[data['stay_date'] > datetime.date(2018,5,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [7]:
# 실제로 실행하는 코드
url = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d9452203-Reviews-Shilla_Stay_Gwanghwamun-Seoul.html"
url2 = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d8118189-Reviews-or240-Shilla_Stay_Mapo-Seoul.html"
url3 = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d7054545-Reviews-Shilla_Stay_Yeoksam-Seoul.html"
url4 = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d9454537-Reviews-Shilla_Stay_Guro-Seoul.html"

data_sample = hotel_review_crawling(url4)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지
16 번째 페이지
17 번째 페이지
18 번째 페이지
19 번째 페이지
20 번째 페이지
21 번째 페이지
22 번째 페이지
23 번째 페이지
24 번째 페이지
25 번째 페이지
26 번째 페이지
27 번째 페이지
28 번째 페이지
29 번째 페이지
30 번째 페이지
31 번째 페이지
32 번째 페이지
33 번째 페이지
34 번째 페이지
35 번째 페이지
36 번째 페이지
37 번째 페이지
38 번째 페이지
39 번째 페이지
40 번째 페이지
41 번째 페이지
42 번째 페이지
43 번째 페이지
44 번째 페이지
45 번째 페이지
46 번째 페이지
47 번째 페이지
48 번째 페이지
49 번째 페이지
50 번째 페이지
51 번째 페이지
52 번째 페이지
53 번째 페이지
54 번째 페이지
55 번째 페이지
56 번째 페이지
57 번째 페이지
58 번째 페이지
59 번째 페이지
60 번째 페이지
61 번째 페이지
62 번째 페이지
63 번째 페이지
64 번째 페이지
65 번째 페이지
66 번째 페이지
67 번째 페이지
68 번째 페이지
69 번째 페이지
70 번째 페이지
71 번째 페이지
72 번째 페이지
73 번째 페이지


,nickname,score,review,stay_date
0,Jae,50,작은 테이블도 있어서 뭐 먹기에도 좋구요\n\n어메니티도 구비잘되어 있어요\n\n적...,2021-07-01
1,ming,40,생각보다 조금 작지만 침대가 푹신하고 역이랑 편의점이 가까워서 좋아요😊😊 에어컨 빵...,2021-07-01
2,BBBJJJHHH,50,7월 중순에 4박정도 이용했습니다.\n신라스테이를 고른 이유는 평균 1박에 8만원 ...,2021-07-01
3,Leg,50,"볼일이 있어 기대 없이 호텔을 예약을 하고 왔는데 정말 잘 쉬다 가네요 , 시국이 ...",2021-07-01
4,eunseo,50,2박 하면서 불편한 점 없이 너무 잘 쉬다 갑니다! 직원분들도 너무 친절하셨고 룸 ...,2021-06-01
...,...,...,...,...
360,jun651,40,적당한 가격과 객실상태도 만족하엿습니다. 어메니티도 좋았구요^^ㅎㅎㅎ\n주변에 식당...,2019-01-01
361,무호2,50,새배드림 패키지로 왔다. 호텔 가격이 저렴하고 기념품으로 곰인형을 주었다. 점심 저...,2019-02-01
362,SHL,10,"호텔 시설은 모던하고 깔끔한데, 관리가 전혀 안됨. 하우스키퍼 퇴근한 6시 이후로는...",2019-02-01
363,1234,10,전에도 이용하고 불친절해서 불쾌했던 곳인데 약속장소와 가까워서 다시 이용해보니 여전...,2019-02-01


In [7]:
data_sample = pd.read_csv('./data//hotel_review_shillastay_mapo_del_emoji.csv')
data_sample = data_sample[['nickname', 'score', 'review', 'stay_date']]

In [8]:
data_sample.stay_date = data_sample.apply(lambda X :datetime.datetime.strptime(X.stay_date,'%Y-%m-%d'), axis=1)

In [9]:
data_sample.stay_date = data_sample.apply(lambda X : X.stay_date.date(), axis=1)

In [10]:
data_sample

,nickname,score,review,stay_date
0,DONGGIL LEE,50,조용하게 편안하게 잘 쉬다갑니다! 교통편도 편리해서 이동하기도 편했습니다. 근처에 ...,2021-06-01
1,Zaen.K,50,객실 내 와이파이랑 유선랜 연결이 잘 돼서 업무 보기 편안합니다. 친절하게 안내 도...,2021-06-01
2,Seung Choi,50,호캉스로 선택해서 마포쪽은 처음 와봤는데 객실이 깨끗하고 구조도 편리하고 좋으네요!...,2021-06-01
3,유빈 권,50,친구랑 호캉스왔는데 고층이라서 뷰가 정말 좋아요. 설명도 친절하게 해주시고 너무 만...,2021-06-01
4,하현오,50,처음 데스크에서 권혁준 선임분이 체크인 해주실때 부터 환한 웃음으로 반겨주셔서 입실...,2021-06-01
...,...,...,...,...
298,jeison88,50,보통 한달에 한 번 일주일 정도 만나는 롱디커플입니다 항상 롯데시티호텔만 이용하다 ...,2019-08-01
299,moon,50,너무 편하게 쉬었고 이벤트 덕분에 편하게 쉬다가 돌아가요 감사합니다!다음에도 방문하...,2019-08-01
300,Minsung choi,40,직원분들도 친절하시고 교통이 편리한 곳에 위치하여 좋았다. 커피 쿠폰을 주셨는데 커...,2019-08-01
301,EUNBIN,50,너무 방 상태도 좋고 직원들이 친절하게 응대 해서 너무 좋습니다 :) 가격대비 방 ...,2019-08-01


In [12]:
# 중간에 크롤링이 끊겼을 경우
url_next = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d8118189-Reviews-or490-Shilla_Stay_Mapo-Seoul.html"
data_sample2 = hotel_review_crawling(url_next)

data_sample_concat = pd.concat([data_sample_concat, data_sample2], axis=0, ignore_index=True)
data_sample_concat

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지


,nickname,score,review,stay_date
0,DONGGIL LEE,50,조용하게 편안하게 잘 쉬다갑니다! 교통편도 편리해서 이동하기도 편했습니다. 근처에 ...,2021-06-01
1,Zaen.K,50,객실 내 와이파이랑 유선랜 연결이 잘 돼서 업무 보기 편안합니다. 친절하게 안내 도...,2021-06-01
2,Seung Choi,50,호캉스로 선택해서 마포쪽은 처음 와봤는데 객실이 깨끗하고 구조도 편리하고 좋으네요!...,2021-06-01
3,유빈 권,50,친구랑 호캉스왔는데 고층이라서 뷰가 정말 좋아요. 설명도 친절하게 해주시고 너무 만...,2021-06-01
4,하현오,50,처음 데스크에서 권혁준 선임분이 체크인 해주실때 부터 환한 웃음으로 반겨주셔서 입실...,2021-06-01
...,...,...,...,...
508,정건희,50,공덕역주변에 비슷한호텔들이 많이 있지만 저는 항상 신라스테이 마포를 선택합니다.\n...,2018-07-01
509,songchohee,50,친구랑 투숙 할곳을\n찾기위해 고심한 끝에 신라스테이 마포점을 선택했다\n처음투숙했...,2018-07-01
510,Bomoonk,50,일때문에 투숙하게 되었다 근처에 여의도 가깝고 지하철도 가까워서 너무 편했다 객실도...,2018-06-01
511,SOHYUN S,50,"친구와 1박2일 서울여행으로 신라스테이 마포에서 투숙하게 됐는데, 너무 만족스러웠습...",2018-03-01


In [13]:
data_sample_concat

,nickname,score,review,stay_date
0,DONGGIL LEE,50,조용하게 편안하게 잘 쉬다갑니다! 교통편도 편리해서 이동하기도 편했습니다. 근처에 ...,2021-06-01
1,Zaen.K,50,객실 내 와이파이랑 유선랜 연결이 잘 돼서 업무 보기 편안합니다. 친절하게 안내 도...,2021-06-01
2,Seung Choi,50,호캉스로 선택해서 마포쪽은 처음 와봤는데 객실이 깨끗하고 구조도 편리하고 좋으네요!...,2021-06-01
3,유빈 권,50,친구랑 호캉스왔는데 고층이라서 뷰가 정말 좋아요. 설명도 친절하게 해주시고 너무 만...,2021-06-01
4,하현오,50,처음 데스크에서 권혁준 선임분이 체크인 해주실때 부터 환한 웃음으로 반겨주셔서 입실...,2021-06-01
...,...,...,...,...
508,정건희,50,공덕역주변에 비슷한호텔들이 많이 있지만 저는 항상 신라스테이 마포를 선택합니다.\n...,2018-07-01
509,songchohee,50,친구랑 투숙 할곳을\n찾기위해 고심한 끝에 신라스테이 마포점을 선택했다\n처음투숙했...,2018-07-01
510,Bomoonk,50,일때문에 투숙하게 되었다 근처에 여의도 가깝고 지하철도 가까워서 너무 편했다 객실도...,2018-06-01
511,SOHYUN S,50,"친구와 1박2일 서울여행으로 신라스테이 마포에서 투숙하게 됐는데, 너무 만족스러웠습...",2018-03-01


In [14]:
# 기본 전처리 전 원본 데이터 저장(유실방지)
data_sample_concat.to_csv('./data/hotel_review_shillastay_gwanghwamun_origin.csv', encoding="utf-8-sig")

In [54]:
data_sample_concat.apply(lambda X : print(type(X.stay_date)), axis=1)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

0      None
1      None
2      None
3      None
4      None
       ... 
763    None
764    None
765    None
766    None
767    None
Length: 768, dtype: object

In [57]:
data_sample_concat.apply(lambda X: datetime.datetime.strptime(X.stay_date,'%Y-%m-%d')
                                      if (str(type(X.stay_date)) != "<class 'datetime.date'>"), axis=1)

SyntaxError: invalid syntax (<ipython-input-57-403685d6f9ea>, line 1)

In [61]:
ds_tmp = data_sample_concat[623:]

In [63]:
ds_tmp.apply(lambda X : print(type(X.stay_date)), axis=1)

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime

623    None
624    None
625    None
626    None
627    None
       ... 
763    None
764    None
765    None
766    None
767    None
Length: 145, dtype: object

In [70]:
data_sample_concat = pd.concat([data_sample, ds_tmp], axis=0, ignore_index=True)
data_sample_concat

,nickname,score,review,stay_date
0,Stella,40,시험때문에 서울 올라가서 신라스테이에서 잤다 처음 방에 들어갔을때는 책상이 작고 욕...,2021-07-01
1,Junhyeok Kang,50,침대가 푹신푹신하고 습도도 낮고 쾌적했습니다 욕조도 좋았고 절이 보이는 뷰도 좋았습...,2021-07-01
2,Yum,50,아이둘과 방문했어요 생각했던 것보다 깔끔하고 생각했던것 보다 단정하고 기대했던 것 ...,2021-07-01
3,YHYS,50,깨끗하고 경치가좋아요 위치도 근처여행하기정말좋아요 경복궁투어하기도좋고 지하철역도가깝...,2021-07-01
4,Kwon,50,위치가 좋아서 잘 머물다갑니다 근처에 맛집도 많고 뷰도 나름 괜찮았어요 다음에도 방...,2021-07-01
...,...,...,...,...
763,Avillasboas,50,약 6개월 만에 다시 재방문..여전히 호텔의 청결도와 위치는 물론 서비스와 친절도 ...,2018-06-01
764,WooYoung L,50,신라스테이 광화문점 이번에 처음 방문해봤는데 다른지점보다 위치도 좋고 깨끗했어요!!...,2018-07-01
765,cgh2245,50,서울 놀러와서 첫날 숙소가 신라스테이였는데 위치적으로도 좋았고요 처음 호텔 들어서면...,2018-04-01
766,hyunwoo,40,방크기는 작지만 조식이 만족스러웠다 예약시 전화를 안받아서 애로사항이 있었지만 그외...,2018-06-01


In [15]:
data = preprocessing_data(data_sample_concat)

In [16]:
data

,nickname,score,review,stay_date
0,DONGGIL LEE,50,조용하게 편안하게 잘 쉬다갑니다! 교통편도 편리해서 이동하기도 편했습니다. 근처에 ...,2021-06-01
1,예진,50,친구랑 휴가차 놀러왔는데 너무 친절하시고 빠른 응대에 너무 좋았어요! 처음 묵어보는...,2021-06-01
2,몬스터 빅,50,가성비좋는가격과 주변교통인프라가 좋아 자주 이용하는곳입니다. 공항 도시철도 다 가깝...,2021-06-01
3,Kwak MoonYoung,50,"조식 포함 패키지로 이용했습니다. 다른 지점들도 마찬가지지만 마포지점 역시 접근성,...",2021-06-01
4,은경 성,50,친구들과 함께 하루를 재밌게 지내려고 투숙했습니다 체크인 시간이 굉장히 길지만 그시...,2021-06-01
...,...,...,...,...
504,vvelyy,40,마포에 비즈니스가 있어 롯데시티랑 고민하다 갔는데 분위기도 그렇고 깔끔하고 좋았어요...,2018-06-01
505,Bomoonk,50,일때문에 투숙하게 되었다 근처에 여의도 가깝고 지하철도 가까워서 너무 편했다 객실도...,2018-06-01
506,Jkyuan,40,역에서 가까워서 이동하기에도 좋고 조식에 쌀국수랑 오믈렛 완젼 짱짱 맛있었어요~!!...,2018-06-01
507,국원 박,50,가격 때문에 선택하는 곳이지만 매번 만족 하고 돌아갑니다// 마포는 이번이 두번째 ...,2018-06-01


In [17]:
# csv 저장을 위한 date to str convert
data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))
# 기본 전처리후 데이터 csv 파일로 저장 
#data.to_csv('./data/hotel_review_shillastay_yeoksam.csv', encoding="utf-8-sig")

<ipython-input-17-7183346d9643>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))


In [18]:
# 리뷰 내 이모지 제거
data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))
#data.to_csv('./data/hotel_review_shillastay_yeoksam_del_emoji.csv', encoding="utf-8-sig")

<ipython-input-18-b71e843350f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))


In [19]:
# 리뷰 내 특수문자 제거
test_data = data
special = re.compile(r'[^ A-Za-z0-9가-힣+]')
test_data['review'] = [special.sub('', s) for s in test_data['review']]

# 특수문자 제거 후 파일 저장
test_data.to_csv('./data/hotel_review_shillastay_mapo_final.csv', encoding="utf-8-sig")

<ipython-input-19-12bd5899dae3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['review'] = [special.sub('', s) for s in test_data['review']]


In [76]:
test_data

,nickname,score,review,stay_date
0,Stella,40,시험때문에 서울 올라가서 신라스테이에서 잤다 처음 방에 들어갔을때는 책상이 작고 욕...,2021-07-01
1,Yum,50,아이둘과 방문했어요 생각했던 것보다 깔끔하고 생각했던것 보다 단정하고 기대했던 것 ...,2021-07-01
2,YHYS,50,깨끗하고 경치가좋아요 위치도 근처여행하기정말좋아요 경복궁투어하기도좋고 지하철역도가깝...,2021-07-01
3,Kwon,50,위치가 좋아서 잘 머물다갑니다 근처에 맛집도 많고 뷰도 나름 괜찮았어요 다음에도 방...,2021-07-01
4,드리머,50,3번째 방문했습니다 근처 맛집들이 정말 많고 룸 컨디션도 좋습니다 왠만한 곳은 걸어...,2021-07-01
...,...,...,...,...
757,hyunwoo,40,방크기는 작지만 조식이 만족스러웠다 예약시 전화를 안받아서 애로사항이 있었지만 그외...,2018-06-01
758,석현 정,50,일 때문에 하루 숙박하게 되었는데 프론트 직원분들 너무 친절하고 좋습니다 특히 yu...,2018-06-01
759,shforyou91,50,일단 광화문이라 위치가 어디 다른곳으로 움직이기도 편하고 너 무 좋았어요 전체적으로...,2018-06-01
760,광석 최,50,여자친구랑 기념일 축하할겸 예약 했던 신라스테이 광화문 일단 위치도 너무 좋았고 ...,2018-06-01
